In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import seaborn as sns
import os
import matplotlib.pyplot as plt
from scipy import stats 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib
import sklearn

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

import lightgbm as lgb
from fbprophet import Prophet
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error

#importing a scoring metric to compare methods
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [2]:
train_features = pd.read_csv("../data/processed/train_features.csv")
train_labels = pd.read_csv("../data/processed/train_labels.csv")
test_features = pd.read_csv("../data/processed/test_features.csv")
test_labels = pd.read_csv("../data/processed/test_labels.csv")

In [3]:
######################## BEGINNING BOOSTING MODEL ##########################
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier

In [4]:
myfavoritenumber = 13
seed = myfavoritenumber
np.random.seed(seed)

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)

In [5]:
X = train_features
y = train_labels

In [6]:
X.set_index(['measurement_date', 'user_code'], inplace=True) #reseting the index
y.set_index(['measurement_date', 'user_code'], inplace=True) #reseting the index

In [7]:
mod = bdt.fit(X, y.values.ravel()) #.ravel converts from col vec to 1d array

In [8]:
mod

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=200)

In [9]:
X_test = test_features
y_test = test_labels

In [10]:
X_test.set_index(['measurement_date', 'user_code'], inplace=True) #reseting the index
y_test.set_index(['measurement_date', 'user_code'], inplace=True) #reseting the index

In [11]:
bdt_predictions = mod.predict(X_test)
bdt_probs = mod.predict_proba(X_test)[:,1]

In [12]:
roc_value = roc_auc_score(y_test, bdt_probs)
roc_value

0.8639918051708217

In [13]:
from sklearn.metrics import confusion_matrix
results = confusion_matrix(y_test, bdt_predictions) 

In [14]:
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test, bdt_predictions)) 
print('Report : ')
print(classification_report(y_test, bdt_predictions))

Confusion Matrix :
[[537  71]
 [ 60  54]]
Accuracy Score : 0.8185595567867036
Report : 
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       608
           1       0.43      0.47      0.45       114

    accuracy                           0.82       722
   macro avg       0.67      0.68      0.67       722
weighted avg       0.83      0.82      0.82       722

